# Library Import


In [1]:
import huggingface_hub as hf # For downloading data from Hugging Face
import pandas as pd # For reading and importing data files into DataFrames used to represent the data


In [2]:
import sys

sys.set_int_max_str_digits(100000)

In [3]:
INPUT_PATH = './prompts'

# Data Import


In [4]:
# TACO Dataset
# Test Files
test_filenames = [
  'ALL/test-00000-of-00001.parquet',
] 

# Train Files
train_filenames = [
  'ALL/train-00001-of-00009.parquet',
  'ALL/train-00002-of-00009.parquet',
  'ALL/train-00003-of-00009.parquet',
  'ALL/train-00004-of-00009.parquet',
  'ALL/train-00005-of-00009.parquet',
  'ALL/train-00006-of-00009.parquet',
  'ALL/train-00007-of-00009.parquet',
  'ALL/train-00008-of-00009.parquet',
]

In [5]:
train_df = pd.DataFrame([])
test_df = pd.DataFrame([])

for filename in train_filenames: # iterate over train_filenames
  path = hf.hf_hub_download( # download dataset
    repo_id='BAAI/TACO', # repo id that contains the files
    repo_type='dataset', # the type should be dataset as the download involves data files
    filename=filename, # the specific filename to be downloaded
  )

  if train_df.empty: # if train_df is still empty
    train_df = pd.read_parquet(path=path) # create a new DataFrame with data instead
  else: # if train_df is not empty
    train_df = pd.concat([train_df, pd.read_parquet(path)], axis=0) # concat the new data along axis=0 so not to create new columns,
                                                                    # instead concat the data as rows

for filename in test_filenames: # iterate over test_filenames
  path = hf.hf_hub_download( # download dataset
    repo_id='BAAI/TACO', # repo id that contains the files
    repo_type='dataset', # the type should be dataset as the download involves data files
    filename=filename, # the specific filename to be downloaded
  )

  if test_df.empty: # if test_df is still empty
    test_df = pd.read_parquet(path=path) # create a new DataFrame with data instead
  else: # if test_df is not empty
    test_df = pd.concat([test_df, pd.read_parquet(path)], axis=0) # concat the new data along axis=0 so not to create new columns,
                                                                    # instead concat the data as rows

# Initial Inspection


train_df and test_df are going to be combined as the project does not involve model testing. This means that there is no need to split the data into training set and testing set. It is more favorable to use all problem sets at once to cover more possibilities and variations.


In [6]:
full_df = pd.concat([train_df, test_df], axis=0) # create a concatenated train_df and test_df

In [7]:
full_df.head()

,question,solutions,starter_code,input_output,difficulty,raw_tags,name,source,tags,skill_types,url,Expected Auxiliary Space,time_limit,date,picture_num,memory_limit,Expected Time Complexity
0,"For every string given as input, you need to t...",[],,"{""inputs"": [""1\naab"", ""3\naab\ndddd\nthisisapa...",UNKNOWN_DIFFICULTY,[],subpalindrome-4,hackerearth,[],[],None,None,None,None,None,None,None
1,A scientist discovered a strange variation of ...,[],,"{""inputs"": [""3\n0\n7\n15\n655\n2711\n6395\n719...",UNKNOWN_DIFFICULTY,[],None,aizu,[],[],None,None,8.0 seconds,None,None,134.217728 megabytes,None
2,Two integers A and B are the inputs. Write a p...,"[""def GCD(x, y):\n\twhile y:\n\t\t(x, y) = (y,...",,"{""inputs"": [[""3"", ""120 140"", ""10213 312"", ""10 ...",EASY,"['LCM', 'Mathematics', 'Algorithms', 'Number T...",None,codechef,"['Mathematics', 'Number theory', 'Implementati...",[],https://www.codechef.com/problems/FLOW016,None,1 seconds,2015-04-27,0,50000 bytes,None
3,Limak is a grizzly bear who desires power and ...,"[""n = int(input())\nv = list(map(int, input()....",,"{""inputs"": [""5\n5 1 11 2 8\n"", ""4\n1 8 8 8\n"",...",EASY,"['greedy', 'implementation']",None,codeforces,"['Implementation', 'Greedy algorithms']",['Greedy algorithms'],https://codeforces.com/problemset/problem/574/A,None,None,2019-12-31,None,None,None
4,"problem\n\nJOI took six subjects: physics, che...","[""lst = []\nfor i in range(6):\n\tn = int(inpu...",,"{""inputs"": [""100\n34\n76\n67\n10\n0"", ""100\n34...",UNKNOWN_DIFFICULTY,[],None,aizu,[],[],None,None,8.0 seconds,None,None,268.435456 megabytes,None


In [8]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23616 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   question                  23616 non-null  object
 1   solutions                 23616 non-null  object
 2   starter_code              23616 non-null  object
 3   input_output              23616 non-null  object
 4   difficulty                23616 non-null  object
 5   raw_tags                  23616 non-null  object
 6   name                      3352 non-null   object
 7   source                    23616 non-null  object
 8   tags                      23616 non-null  object
 9   skill_types               23616 non-null  object
 10  url                       18915 non-null  object
 11  Expected Auxiliary Space  2386 non-null   object
 12  time_limit                10519 non-null  object
 13  date                      6204 non-null   object
 14  picture_num               671

All column type is object, can be string or other data structures like list, tuple, or dictionary


In [9]:
eda_df = full_df.copy()

# Utility Functions


### Check condition for any empty row


In [10]:
from typing import Tuple

In [11]:
def empty_rows(data: pd.Series) -> Tuple[pd.Series, int]:
  rows = []
  for row in data:
    if not row or not len(row):
      rows.append(row)

  return (pd.Series(rows), len(rows))

### Check null or empty rows


In [12]:
def null_and_empty_rows(data: pd.Series) -> None:

  empty = empty_rows(data)
  print(f'Null count: {data.isnull().sum()} rows.')
  print(f'Row containing empty data: {empty[1]} rows.')
  print(f'Empty portion {round((empty[1] / data.shape[0]) * 100, 2)}%')

### Check detailed data type


In [13]:
def check_type(data: pd.Series) -> None:
  for row in data:
    if row:
      return type(row)

### First n Non Null Values


In [14]:
def n_non_null_head(data: pd.Series, n: int = 5) -> pd.Series:
  head = []
  for row in data:
    if len(head) >= n:
      break
    if row:
      head.append(row)

  if len(head) < n:
    head.extend([None] * (n - len(head)))
  
  return pd.Series(head)

### Initial Check


In [15]:
def initial_check(data: pd.Series) -> None:
  column_type = check_type(data)
  print(f'Type: {column_type}\n')
  print('First 5 non-null rows:')
  print(n_non_null_head(data))

  print('\nNull and empty values:')
  null_and_empty_rows(data)

### Decode Data Structures From a String


In [16]:
from json import JSONDecodeError
import json

def safe_decoding(row: str) -> any:
  try:
    if len(row) and row:
      return json.loads(row.replace('\'', '\"'))
    else:
      return row
  except JSONDecodeError:
    return row

def decode_json(data: pd.Series) -> pd.Series:
  return data.apply(lambda row: safe_decoding(row))

# Column Analysis


## 1. Question


In [17]:
initial_check(eda_df['question'])

Type: <class 'str'>

First 5 non-null rows:
0    For every string given as input, you need to t...
1    A scientist discovered a strange variation of ...
2    Two integers A and B are the inputs. Write a p...
3    Limak is a grizzly bear who desires power and ...
4    problem\n\nJOI took six subjects: physics, che...
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


Questions are in string, so no need to decode the data


The question column is essential as it holds the actual problem set questions. It seems that the there is no null row too, which means that all rows can be used for prompting.


## 2. Solutions


In [18]:
initial_check(eda_df['solutions'])

Type: <class 'str'>

First 5 non-null rows:
0                                                   []
1                                                   []
2    ["def GCD(x, y):\n\twhile y:\n\t\t(x, y) = (y,...
3    ["n = int(input())\nv = list(map(int, input()....
4    ["lst = []\nfor i in range(6):\n\tn = int(inpu...
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


The column type is string, but the content is a Python list, which means it needs to be decoded.


Some solutions are empty and the solution column itself cannot be used as a benchmark to the chatbot response as it is difficult to quantify how close the chabot codes are to these solutions and how good the codes based on the solutions data.


## 3. Starter Code


In [19]:
initial_check(eda_df['starter_code'])

Type: <class 'str'>

First 5 non-null rows:
0    #User function Template for python3\n\n\n\n'''...
1    class Solution:\n    def minSumOfLengths(self,...
2    from typing import List\n\n\n\n\n\n\n\nclass S...
3    #User function Template for python3\n\nclass S...
4    #User function Template for python3\n\n# Retur...
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 18303 rows.
Empty portion 77.5%


Starter code does not contain any valuable information, so it can be dropped.


## 4. Input-Output


In [20]:
initial_check(eda_df['input_output'])

Type: <class 'str'>

First 5 non-null rows:
0    {"inputs": ["1\naab", "3\naab\ndddd\nthisisapa...
1    {"inputs": ["3\n0\n7\n15\n655\n2711\n6395\n719...
2    {"inputs": [["3", "120 140", "10213 312", "10 ...
3    {"inputs": ["5\n5 1 11 2 8\n", "4\n1 8 8 8\n",...
4    {"inputs": ["100\n34\n76\n67\n10\n0", "100\n34...
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


The data type is string, but the content is dictionary, so it needs to be decoded.


In [21]:
# To get better representation on how the data is structured, decoding must be done.
eda_df['input_output'] = decode_json(eda_df['input_output'])

In [22]:
initial_check(eda_df['input_output'])

Type: <class 'dict'>

First 5 non-null rows:
0    {'inputs': ['1
aab', '3
aab
dddd
thisisapalind...
1    {'inputs': ['3
0
7
15
655
2711
6395
7195
8465
...
2    {'inputs': [['3', '120 140', '10213 312', '10 ...
3    {'inputs': ['5
5 1 11 2 8
', '4
1 8 8 8
', '2
...
4    {'inputs': ['100
34
76
67
10
0', '100
34
76
96...
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


Looking at the data, each input output data contains a dictionary with:<br />

1. Two keys: inputs and outputs
2. The values are a list of inputs and outputs with each index of input attributes to the corresponding index of the output


## 5. Difficulty


In [23]:
initial_check(eda_df['difficulty'])

Type: <class 'str'>

First 5 non-null rows:
0    UNKNOWN_DIFFICULTY
1    UNKNOWN_DIFFICULTY
2                  EASY
3                  EASY
4    UNKNOWN_DIFFICULTY
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


In [24]:
eda_df['difficulty'].value_counts()

difficulty
EASY                  8099
UNKNOWN_DIFFICULTY    4467
MEDIUM                3082
HARD                  2995
MEDIUM_HARD           2656
VERY_HARD             2317
Name: count, dtype: int64

UNKNOWN_DIFFICULTY will not be used as it will be more difficult to filter out questions and analyze the questions and answers later on.


UNKNOWN_DIFFICULTY should be excluded as it will be difficult to discriminate between problem sets later on.


## 6. Raw Tags


In [25]:
initial_check(eda_df['raw_tags'])

Type: <class 'str'>

First 5 non-null rows:
0                                                   []
1                                                   []
2    ['LCM', 'Mathematics', 'Algorithms', 'Number T...
3                         ['greedy', 'implementation']
4                                                   []
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


Some tags are empty, but keeping empty tags are not going to impact the problem set. This is why initial check says there are no empty rows even though some tags are empty.


The data needs to be decoded as they are list inside of a string.


In [26]:
eda_df['raw_tags'] = decode_json(eda_df['raw_tags'])

In [27]:
initial_check(eda_df['raw_tags'])

Type: <class 'list'>

First 5 non-null rows:
0    [LCM, Mathematics, Algorithms, Number Theory, ...
1                             [greedy, implementation]
2                               [math, implementation]
3    [combinatorics, matrices, math, constructive a...
4                             [greedy, implementation]
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 6874 rows.
Empty portion 29.11%


6874 of the total rows do not empty raw tags list.


## 7. Name


In [28]:
initial_check(eda_df['name'])

Type: <class 'str'>

First 5 non-null rows:
0                             subpalindrome-4
1                             modified-number
2                                   leading-1
3          miss-dd-and-her-mysterious-numbers
4    AtCoder Beginner Contest 083 - Some Sums
dtype: object

Null and empty values:
Null count: 20264 rows.
Row containing empty data: 20264 rows.
Empty portion 85.81%


Most of name column values are null, around 85.8062%, so this can be dropped as most data will have null name while the others have name.


## 8. Source


In [29]:
initial_check(eda_df['source'])

Type: <class 'str'>

First 5 non-null rows:
0    hackerearth
1           aizu
2       codechef
3     codeforces
4           aizu
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


In [30]:
sources_set = set()

for source in eda_df['source']:
  sources_set.add(source)

print(sources_set)

{'codewars', 'geeksforgeeks', 'hackerearth', 'leetcode', 'atcoder', 'codechef', 'codeforces', 'kattis', 'hackerrank', 'aizu'}


These are the source of the problem sets. This output can be used to analyze the terms of services of each problem sets, and what can be used for this research.


TOS agreement:

- Code Forces https://codeforces.com/terms 👌
- LeetCode https://leetcode.com/terms/ ❌ (likely no)
- geeksforgeeks ❌ (likely no)
- kattis https://open.kattis.com/info/tos 👌Likely okay (non-commercial)
- aizu https://judge.u-aizu.ac.jp/onlinejudge/submission_note.jsp ❌ Risky (no clear terms)
- hackerearth https://www.hackerearth.com/terms-of-service/ ❌ High risk
- atcoder https://atcoder.jp/tos 👌Yes (Citation Required)
- Codewars https://www.codewars.com/about/terms-of-service 👌Yes (Citation Required)
- hackerrank https://www.hackerrank.com/terms-of-service/ 👌Yes (Citation Required, no problemset reproduction)
- codechef https://www.codechef.com/terms 👌Likely okay (non-commercial


This column is not used specifically in the prompting, so it can be dropped later.


## 9. Tags


In [31]:
initial_check(eda_df['tags'])

Type: <class 'str'>

First 5 non-null rows:
0                                                   []
1                                                   []
2    ['Mathematics', 'Number theory', 'Implementati...
3              ['Implementation', 'Greedy algorithms']
4                                                   []
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


Tags are supposed to be a list, so it needs to be decoded.


In [32]:
eda_df['tags'] = decode_json(eda_df['tags'])

In [33]:
initial_check(eda_df['tags'])

Type: <class 'list'>

First 5 non-null rows:
0         [Mathematics, Number theory, Implementation]
1                  [Implementation, Greedy algorithms]
2                        [Mathematics, Implementation]
3    [Matrices, Combinatorics, Mathematics, Constru...
4                  [Implementation, Greedy algorithms]
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 7652 rows.
Empty portion 32.4%


7652 tags are empty, but it will not be dropped as problem sets with no tags are still included.


## 10. Skill Types


In [34]:
initial_check(eda_df['skill_types'])

Type: <class 'str'>

First 5 non-null rows:
0                       []
1                       []
2                       []
3    ['Greedy algorithms']
4                       []
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 0 rows.
Empty portion 0.0%


Skill Types is structured as a list, but the type is string, so it needs to be decoded.


In [35]:
eda_df['skill_types'] = decode_json(eda_df['skill_types'])

In [36]:
initial_check(eda_df['skill_types'])

Type: <class 'list'>

First 5 non-null rows:
0                                  [Greedy algorithms]
1                                  [Greedy algorithms]
2                     [Data structures, Range queries]
3                                    [Complete search]
4    [Dynamic programming, Amortized analysis, Sort...
dtype: object

Null and empty values:
Null count: 0 rows.
Row containing empty data: 13596 rows.
Empty portion 57.57%


13596 list of the entire skill types lists are empty, these empty skill type lists will be explained on the prompt construction.


## 11. URL


In [37]:
initial_check(eda_df['url'])

Type: <class 'str'>

First 5 non-null rows:
0           https://www.codechef.com/problems/FLOW016
1     https://codeforces.com/problemset/problem/574/A
2      https://codeforces.com/problemset/problem/40/C
3    https://codeforces.com/problemset/problem/1332/E
4     https://codeforces.com/problemset/problem/374/A
dtype: object

Null and empty values:
Null count: 4701 rows.
Row containing empty data: 4701 rows.
Empty portion 19.91%


URL information itself will not affect the produced solutions because they do not correlate with the problem sets, so it will be dropped for the prompting.


## 12. Expected Auxiliary Space


In [38]:
initial_check(eda_df['Expected Auxiliary Space'])

Type: <class 'str'>

First 5 non-null rows:
0                            O(1).
1                         O(N^{2})
2                             O(1)
3    O(Height of the Binary Tree).
4                 O(|str1|*|str2|)
dtype: object

Null and empty values:
Null count: 21230 rows.
Row containing empty data: 21352 rows.
Empty portion 90.41%


In [39]:
eda_df['Expected Auxiliary Space'].value_counts()

Expected Auxiliary Space
O(1)                             961
O(N)                             307
O(1).                            214
                                 122
O(n)                             103
                                ... 
O(N * WORD_LEN).                   1
O(Distinct Characters).            1
O(1) for all the 5 functions.      1
O(N^{2}*K)                         1
O(Height of the Binary Tree).      1
Name: count, Length: 305, dtype: int64

Around 90% of problem sets do not have expected auxiliary space, so they will be assumed to not have expectations, which will be mentioned in the prompts later.


## 13. Time Limit


In [40]:
initial_check(eda_df['time_limit'])

Type: <class 'str'>

First 5 non-null rows:
0    8.0 seconds
1      1 seconds
2    8.0 seconds
3    2.0 seconds
4      2 seconds
dtype: object

Null and empty values:
Null count: 13097 rows.
Row containing empty data: 13097 rows.
Empty portion 55.46%


Around 55.46% of problem sets do not have time limit, so they will be assumed to not have expectations, which will be mentioned in the prompts later.


## 14. Date


In [41]:
initial_check(eda_df['date'])

Type: <class 'str'>

First 5 non-null rows:
0    2015-04-27
1    2019-12-31
2    2020-03-31
3    2019-12-31
4    2019-12-31
dtype: object

Null and empty values:
Null count: 17412 rows.
Row containing empty data: 17412 rows.
Empty portion 73.73%


Date information itself will not affect the produced solutions because they do not correlate with the problem sets, so it will be dropped altogether.


## 15. Picture Num


In [42]:
initial_check(eda_df['picture_num'])

Type: <class 'str'>

First 5 non-null rows:
0    0
1    1
2    0
3    0
4    0
dtype: object

Null and empty values:
Null count: 16906 rows.
Row containing empty data: 16906 rows.
Empty portion 71.59%


In [43]:
eda_df[['url', 'picture_num']].loc[eda_df['url'] == None]

,url,picture_num


Picture Num is dependent on url. If URL exists, picture_num can then exist. This indicate that picture num points to the picture inside of the URL if picture_num exists. This can be useful for feature engineering.


## 16. Memory Limit


In [44]:
initial_check(eda_df['memory_limit'])

Type: <class 'str'>

First 5 non-null rows:
0    134.217728 megabytes
1             50000 bytes
2    268.435456 megabytes
3         256.0 megabytes
4           512 megabytes
dtype: object

Null and empty values:
Null count: 13096 rows.
Row containing empty data: 13096 rows.
Empty portion 55.45%


55.45% of memory limit is empty. These empty values will be assumed as no time limit is given for the problem sets.


## 17. Expected Time Complexity


In [45]:
initial_check(eda_df['Expected Time Complexity'])

Type: <class 'str'>

First 5 non-null rows:
0               O(N).
1            O(N^{2})
2              O(|S|)
3               O(N).
4    O(|str1|*|str2|)
dtype: object

Null and empty values:
Null count: 21013 rows.
Row containing empty data: 21135 rows.
Empty portion 89.49%


55.45% of expected time complexity is empty. These empty values will be assumed as no expected time complexity is given for the problem sets.


# Feature Engineering


In [46]:
cleaned_df = full_df.copy()

### Drop Columns


In [ ]:
# cleaned_df.drop(columns=['solutions', 'starter_code', 'name', 'date'], inplace=True, axis=1)

### Filter Out Rows


In [48]:
cleaned_df = cleaned_df.loc[cleaned_df['difficulty'] != 'UNKNOWN_DIFFICULTY']

### Decoding


In [49]:
decoded_cols = ['input_output', 'raw_tags', 'tags', 'skill_types']

for col in decoded_cols:
  cleaned_df[col] = decode_json(cleaned_df[col])

### Results


In [50]:
cleaned_df.reset_index(inplace=True, drop=True)

In [51]:
cleaned_df.head()

,question,input_output,difficulty,raw_tags,source,tags,skill_types,url,Expected Auxiliary Space,time_limit,picture_num,memory_limit,Expected Time Complexity
0,Two integers A and B are the inputs. Write a p...,"{'inputs': [['3', '120 140', '10213 312', '10 ...",EASY,"[LCM, Mathematics, Algorithms, Number Theory, ...",codechef,"[Mathematics, Number theory, Implementation]",[],https://www.codechef.com/problems/FLOW016,None,1 seconds,0,50000 bytes,None
1,Limak is a grizzly bear who desires power and ...,"{'inputs': ['5 5 1 11 2 8 ', '4 1 8 8 8 ', '2 ...",EASY,"[greedy, implementation]",codeforces,"[Implementation, Greedy algorithms]",[Greedy algorithms],https://codeforces.com/problemset/problem/574/A,None,None,None,None,None
2,Last year the world's largest square was built...,"{'inputs': ['99085 7738 98097 -6487 ', '1 2 2 ...",HARD,"[math, implementation]",codeforces,"[Mathematics, Implementation]",[],https://codeforces.com/problemset/problem/40/C,None,2.0 seconds,None,256.0 megabytes,None
3,Alice has got addicted to a game called Sirtet...,"{'inputs': ['2 2 1 1 ', '1 2 1 2 ', '485 117 3...",HARD,"[combinatorics, matrices, math, constructive a...",codeforces,"[Matrices, Combinatorics, Mathematics, Constru...",[],https://codeforces.com/problemset/problem/1332/E,None,2 seconds,1,512 megabytes,None
4,Dima and Inna are doing so great! At the momen...,"{'inputs': ['5 7 1 3 2 2 ', '5 5 2 3 1 1 ', '1...",HARD,"[greedy, implementation]",codeforces,"[Implementation, Greedy algorithms]",[Greedy algorithms],https://codeforces.com/problemset/problem/374/A,None,None,None,None,None


In [52]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19149 entries, 0 to 19148
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   question                  19149 non-null  object
 1   input_output              19149 non-null  object
 2   difficulty                19149 non-null  object
 3   raw_tags                  19149 non-null  object
 4   source                    19149 non-null  object
 5   tags                      19149 non-null  object
 6   skill_types               19149 non-null  object
 7   url                       17891 non-null  object
 8   Expected Auxiliary Space  2386 non-null   object
 9   time_limit                7915 non-null   object
 10  picture_num               6710 non-null   object
 11  memory_limit              7915 non-null   object
 12  Expected Time Complexity  2603 non-null   object
dtypes: object(13)
memory usage: 1.9+ MB


In [53]:
for col in cleaned_df:
  print(f'[{col}]\'s type: {cleaned_df[col].dtype}')

[question]'s type: object
[input_output]'s type: object
[difficulty]'s type: object
[raw_tags]'s type: object
[source]'s type: object
[tags]'s type: object
[skill_types]'s type: object
[url]'s type: object
[Expected Auxiliary Space]'s type: object
[time_limit]'s type: object
[picture_num]'s type: object
[memory_limit]'s type: object
[Expected Time Complexity]'s type: object


# Problems Selection


In [54]:
ps_df = cleaned_df.copy()

In [55]:
# Drop rows where the source doesn't approve in the terms of service
tosUnapproved = {
    "geeksforgeeks",
    "leetcode",
    "aizu",
    "hackerearth"
}

# Drop rows where the source doesn't approve in the terms of service
ps_df = ps_df[~ps_df['source'].isin(tosUnapproved)]
print(f"Filtered out {len(cleaned_df) - len(ps_df)} problems with unapproved sources.")
print(f"Remaining problems: {len(ps_df)}")

Filtered out 4326 problems with unapproved sources.
Remaining problems: 14823


In [56]:
# Drop rows that don't have any input or output test cases
filtered_ps_df = ps_df.loc[
    ps_df['input_output'].apply(
        lambda x: isinstance(x, dict) and bool(x.get('inputs')) and bool(x.get('outputs'))
    )
]
ps_df = filtered_ps_df

In [57]:
# Drop rows where skill_types is empty or NaN
# Empty list is considered as empty
ps_df = ps_df[ps_df['skill_types'].apply(lambda x: x is not None and len(x) > 0)]

# Reset index after filtering
ps_df.reset_index(drop=True, inplace=True)

# Display info about the filtered DataFrame
print(f"Rows before filtering: {len(cleaned_df)}")
print(f"Rows after filtering: {len(ps_df)}")
print(f"Filtered out {len(cleaned_df) - len(ps_df)} rows")

Rows before filtering: 19149
Rows after filtering: 7147
Filtered out 12002 rows


In [91]:
randomness = 42
# Max problems per skill type and difficulty
max_counts = {
    'EASY': 18,
    'MEDIUM': 10,
    'MEDIUM_HARD': 8,
    'HARD': 3,
    'VERY_HARD': 3
}

# Get all unique skills from the skill_types column
all_skills = set()
for skills in ps_df['skill_types']:
    if skills and len(skills) > 0:
        all_skills.update(skills)

# Create an empty dataframe to store the results
filtered_df = pd.DataFrame(columns=ps_df.columns)

# Process each skill type
for skill in all_skills:
    # Get problems with this skill
    skill_mask = ps_df['skill_types'].apply(lambda x: skill in x if x else False)
    skill_df = ps_df[skill_mask].copy()
    
    # Sample problems for each difficulty level
    sampled_by_difficulty = pd.DataFrame(columns=ps_df.columns)
    
    for difficulty, max_count in max_counts.items():
        difficulty_df = skill_df[skill_df['difficulty'] == difficulty]
        
        if len(difficulty_df) > 0:
            # Sample up to max_count problems of this difficulty
            sample_size = min(max_count, len(difficulty_df))
            sampled = difficulty_df.sample(n=sample_size, random_state=randomness)
            sampled_by_difficulty = pd.concat([sampled_by_difficulty, sampled])
            
            # Remove the sampled problems from skill_df to avoid duplicates
            # when a problem belongs to multiple skill types
            skill_df = skill_df.loc[~skill_df.index.isin(sampled.index)]
    
    # Add the sampled problems for this skill to the result
    filtered_df = pd.concat([filtered_df, sampled_by_difficulty])
    
# Remove duplicates (in case a problem was selected for multiple skills)
filtered_df = filtered_df.drop_duplicates(subset=[col for col in filtered_df.columns if col not in ['input_output', 'raw_tags', 'tags', 'skill_types']])

# Reset index
filtered_df.reset_index(drop=True, inplace=True)

In [92]:
# Display statistics about the resulting dataset
print(f"Total problems in balanced dataset: {len(filtered_df)}")

skill_difficulty_counts = {}
for skills, difficulty in zip(filtered_df['skill_types'], filtered_df['difficulty']):
    for skill in skills:
        if skill not in skill_difficulty_counts:
            skill_difficulty_counts[skill] = {'EASY': 0, 'MEDIUM': 0, 'HARD': 0, 'MEDIUM_HARD': 0, 'VERY_HARD': 0}
        skill_difficulty_counts[skill][difficulty] += 1

for skill, counts in skill_difficulty_counts.items():
    total_problems = sum(counts.values())
    print(f"\nSkill: {skill} (Total problems: {total_problems})")
    for difficulty, count in counts.items():
        print(f"  {difficulty}: {count}")

print("\nOverall distribution by difficulty:")
print(filtered_df['difficulty'].value_counts())

print("\nDistribution by source:")
print(filtered_df['source'].value_counts())

Total problems in balanced dataset: 301

Skill: Dynamic programming (Total problems: 78)
  EASY: 23
  MEDIUM: 20
  HARD: 8
  MEDIUM_HARD: 17
  VERY_HARD: 10

Skill: Greedy algorithms (Total problems: 100)
  EASY: 42
  MEDIUM: 26
  HARD: 6
  MEDIUM_HARD: 20
  VERY_HARD: 6

Skill: Complete search (Total problems: 84)
  EASY: 35
  MEDIUM: 17
  HARD: 6
  MEDIUM_HARD: 18
  VERY_HARD: 8

Skill: Sorting (Total problems: 100)
  EASY: 35
  MEDIUM: 27
  HARD: 7
  MEDIUM_HARD: 22
  VERY_HARD: 9

Skill: Data structures (Total problems: 86)
  EASY: 32
  MEDIUM: 16
  HARD: 11
  MEDIUM_HARD: 18
  VERY_HARD: 9

Skill: Bit manipulation (Total problems: 49)
  EASY: 20
  MEDIUM: 12
  HARD: 3
  MEDIUM_HARD: 9
  VERY_HARD: 5

Skill: Amortized analysis (Total problems: 58)
  EASY: 21
  MEDIUM: 12
  HARD: 5
  MEDIUM_HARD: 16
  VERY_HARD: 4

Skill: Range queries (Total problems: 22)
  EASY: 5
  MEDIUM: 3
  HARD: 3
  MEDIUM_HARD: 8
  VERY_HARD: 3

Overall distribution by difficulty:
difficulty
EASY           1

In [93]:
filtered_df = filtered_df[:300]
filtered_df.reset_index(drop=True, inplace=True)
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   question                  300 non-null    object
 1   input_output              300 non-null    object
 2   difficulty                300 non-null    object
 3   raw_tags                  300 non-null    object
 4   source                    300 non-null    object
 5   tags                      300 non-null    object
 6   skill_types               300 non-null    object
 7   url                       300 non-null    object
 8   Expected Auxiliary Space  0 non-null      object
 9   time_limit                179 non-null    object
 10  picture_num               137 non-null    object
 11  memory_limit              179 non-null    object
 12  Expected Time Complexity  6 non-null      object
dtypes: object(13)
memory usage: 30.6+ KB


# Prompts Construction


In [94]:
pc_df = filtered_df.copy()

In [ ]:
# pc_df.drop(columns=['picture_num', 'source'], axis=1, inplace=True)

In [96]:
pc_df.head()

,question,input_output,difficulty,raw_tags,tags,skill_types,url,Expected Auxiliary Space,time_limit,memory_limit,Expected Time Complexity
0,You are given a problemset consisting of $n$ p...,"{'inputs': ['10 1 2 5 6 7 10 21 23 24 49 ', '5...",EASY,"[greedy, math, dp]","[Dynamic programming, Mathematics, Greedy algo...","[Dynamic programming, Greedy algorithms]",https://codeforces.com/problemset/problem/1029/B,None,None,None,None
1,"Catherine has a deck of n cards, each of which...","{'inputs': ['2 RB ', '3 GRG ', '5 BBBBB ', '1 ...",EASY,"[math, dp, constructive algorithms]","[Dynamic programming, Mathematics, Constructiv...",[Dynamic programming],https://codeforces.com/problemset/problem/626/B,None,None,None,None
2,Monocarp has been collecting rare magazines fo...,{'inputs': ['4 5 01110 10 5 8 9 6 6 011011 20 ...,EASY,"[greedy, dp, constructive algorithms]","[Greedy algorithms, Dynamic programming, Const...","[Dynamic programming, Greedy algorithms]",https://codeforces.com/problemset/problem/1743/C,None,2 seconds,256 megabytes,None
3,Justin likes to play a game called “Weight of ...,"{'inputs': ['11 131 ', '7 96 ', '26 677 '], 'o...",EASY,"[dp & greedy, dp]",[Dynamic programming],[Dynamic programming],https://open.kattis.com/problems/weightofwords,None,None,None,None
4,Athenaeus has just finished creating his lates...,{'inputs': ['5 6 1 2 2 2 5 6 2 4 4 6 1 1 3 4 4...,EASY,"[greedy, dp]","[Dynamic programming, Greedy algorithms]","[Dynamic programming, Greedy algorithms]",https://codeforces.com/problemset/problem/1466/B,None,1 second,256 megabytes,None


In [97]:
pc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   question                  300 non-null    object
 1   input_output              300 non-null    object
 2   difficulty                300 non-null    object
 3   raw_tags                  300 non-null    object
 4   tags                      300 non-null    object
 5   skill_types               300 non-null    object
 6   url                       300 non-null    object
 7   Expected Auxiliary Space  0 non-null      object
 8   time_limit                179 non-null    object
 9   memory_limit              179 non-null    object
 10  Expected Time Complexity  6 non-null      object
dtypes: object(11)
memory usage: 25.9+ KB


In [114]:
from typing import List

prompts: List[str] = []
input_output: List[dict] = []
time_limit = []
memory_limit = []

for i, val in enumerate(pc_df.values):
  input_output.append(val[1])
  time_limit.append(val[7])
  memory_limit.append(val[8])
  prompts.append(
    'Below is provided a **competitive programming** problem set.\n' \
    'Please solve the problem set with:\n' \
    '1. No pre-assumptions.\n' \
    '2. **Solve using C++.**\n' \
    '3. Only write code comments when necessary (write code comments only in the code block inline).\n' \
    '4. Keeping in mind that the solution can actually reproduce the correct inputs and outputs as will be provided.\n' \
    '5. Only provide solution as the answer without any alternatives **(pick the best of all possibilities with best practices and performance)**.\n' \
    '6. **All explanation MUST be inside of the code itself in the forms of comments.**\n' \
    '7. No additional explanation outside of the code block (your raw output will be compiled, don\'t add anything outside of the code).\n' \
    # '8. Every print out / stdout has to have \'\\n\' at the very end (don\'t add \'\\n\' if the code process already print \'\\n\' at the end, then don\'t print another \'\\n\').\n' \
    '**Question**\n' \
    f'{val[0]}\n' \
    '\n' \
    f'**Difficulty: {val[2]}**\n' \
    '\n' \
    f'**Raw Tags: {val[3]}**\n' \
    '\n' \
    f'**Tags: {val[4]}**\n' \
    '\n' \
    f'**Skill Types: {val[5]}**\n' \
    '\n' \
    f'**Expected Auxiliary Space: {val[7] if val[7] else "No expected aurxiliary space or auxiliary space limit."}**\n' \
    '\n' \
    f'**Time Limit: {val[8] if val[8] else "No time limit."}**\n' \
    '\n' \
    f'**Memory Limit: {val[9] if val[9] else "No memory limit"}**\n' \
    '\n' \
    f'**Expected Time Complexity: {val[10] if val[10] else "No expected time complexity or time complexity limit."}**\n' \
  )

prompts[:5]

["Below is provided a **competitive programming** problem set.\nPlease solve the problem set with:\n1. No pre-assumptions.\n2. **Solve using C++.**\n3. Only write code comments when necessary (write code comments only in the code block inline).\n4. Keeping in mind that the solution can actually reproduce the correct inputs and outputs as will be provided.\n5. Only provide solution as the answer without any alternatives **(pick the best of all possibilities with best practices and performance)**.\n6. **All explanation MUST be inside of the code itself in the forms of comments.**\n7. No additional explanation outside of the code block (your raw output will be compiled, don't add anything outside of the code).\n**Question**\nYou are given a problemset consisting of $n$ problems. The difficulty of the $i$-th problem is $a_i$. It is guaranteed that all difficulties are distinct and are given in the increasing order.\n\nYou have to assemble the contest which consists of some problems of the 

In [ ]:
# final_df = pd.DataFrame()
final_df = pc_df.copy()
final_df['prompts'] = pd.Series(prompts)
final_df['input_output'] = pd.Series(input_output)
final_df['inputs'] = final_df['input_output'].apply(lambda x: x['inputs'] if isinstance(x, dict) else None)
final_df['outputs'] = final_df['input_output'].apply(lambda x: x['outputs'] if isinstance(x, dict) else None)
final_df.drop(columns=['input_output'], inplace=True)
final_df['time_limit'] = pd.Series(time_limit)
final_df['memory_limit'] = pd.Series(memory_limit)
final_df['done'] = pd.Series([False] * final_df.shape[0])

In [116]:
final_df.head()

,question,difficulty,raw_tags,tags,skill_types,url,Expected Auxiliary Space,time_limit,memory_limit,Expected Time Complexity,prompts,inputs,outputs,done
0,You are given a problemset consisting of $n$ p...,EASY,"[greedy, math, dp]","[Dynamic programming, Mathematics, Greedy algo...","[Dynamic programming, Greedy algorithms]",https://codeforces.com/problemset/problem/1029/B,None,None,None,None,Below is provided a **competitive programming*...,"[10\n1 2 5 6 7 10 21 23 24 49\n, 5\n2 10 50 11...","[4\n, 1\n, 3\n, 1\n, 2\n, 4\n, 5\n, 3\n, 7\n, ...",False
1,"Catherine has a deck of n cards, each of which...",EASY,"[math, dp, constructive algorithms]","[Dynamic programming, Mathematics, Constructiv...",[Dynamic programming],https://codeforces.com/problemset/problem/626/B,None,None,None,None,Below is provided a **competitive programming*...,"[2\nRB\n, 3\nGRG\n, 5\nBBBBB\n, 1\nR\n, 200\nB...","[G\n, BR\n, B\n, R\n, BGR\n, BG\n, BGR\n, BGR\...",False
2,Monocarp has been collecting rare magazines fo...,EASY,"[greedy, dp, constructive algorithms]","[Greedy algorithms, Dynamic programming, Const...","[Dynamic programming, Greedy algorithms]",https://codeforces.com/problemset/problem/1743/C,None,None,2 seconds,None,Below is provided a **competitive programming*...,[4\n5\n01110\n10 5 8 9 6\n6\n011011\n20 10 9 3...,"[27\n80\n0\n14\n, 2\n2\n2\n2\n2\n2\n2\n, 80\n8...",False
3,Justin likes to play a game called “Weight of ...,EASY,"[dp & greedy, dp]",[Dynamic programming],[Dynamic programming],https://open.kattis.com/problems/weightofwords,None,None,None,None,Below is provided a **competitive programming*...,"[11 131\n, 7 96\n, 26 677\n]","[programming\n, contest\n, impossible\n]",False
4,Athenaeus has just finished creating his lates...,EASY,"[greedy, dp]","[Dynamic programming, Greedy algorithms]","[Dynamic programming, Greedy algorithms]",https://codeforces.com/problemset/problem/1466/B,None,None,1 second,None,Below is provided a **competitive programming*...,[5\n6\n1 2 2 2 5 6\n2\n4 4\n6\n1 1 3 4 4 5\n1\...,"[5\n2\n6\n1\n3\n, 5\n2\n6\n1\n3\n, 5\n2\n6\n1\...",False


In [117]:
final_df[final_df['inputs'].isna()]

,question,difficulty,raw_tags,tags,skill_types,url,Expected Auxiliary Space,time_limit,memory_limit,Expected Time Complexity,prompts,inputs,outputs,done


In [125]:
# Fix: Set inplace=True to modify final_df directly
final_df["prompts"].duplicated().sum()

np.int64(0)

In [126]:
final_df.index.name = 'id'

In [127]:
final_df.to_csv(f'{INPUT_PATH}/input.csv')

In [128]:
checkdf = pd.read_csv(f'{INPUT_PATH}/input.csv', index_col=0)
checkdf.info()
checkdf.head()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   question                  300 non-null    object 
 1   difficulty                300 non-null    object 
 2   raw_tags                  300 non-null    object 
 3   tags                      300 non-null    object 
 4   skill_types               300 non-null    object 
 5   url                       300 non-null    object 
 6   Expected Auxiliary Space  0 non-null      float64
 7   time_limit                0 non-null      float64
 8   memory_limit              179 non-null    object 
 9   Expected Time Complexity  6 non-null      object 
 10  prompts                   300 non-null    object 
 11  inputs                    300 non-null    object 
 12  outputs                   300 non-null    object 
 13  done                      300 non-null    bool   
dtypes: bool(1), flo

,question,difficulty,raw_tags,tags,skill_types,url,Expected Auxiliary Space,time_limit,memory_limit,Expected Time Complexity,prompts,inputs,outputs,done
id,,,,,,,,,,,,,,
0,You are given a problemset consisting of $n$ p...,EASY,"['greedy', 'math', 'dp']","['Dynamic programming', 'Mathematics', 'Greedy...","['Dynamic programming', 'Greedy algorithms']",https://codeforces.com/problemset/problem/1029/B,NaN,NaN,NaN,NaN,Below is provided a **competitive programming*...,"['10\n1 2 5 6 7 10 21 23 24 49\n', '5\n2 10 50...","['4\n', '1\n', '3\n', '1\n', '2\n', '4\n', '5\...",False
1,"Catherine has a deck of n cards, each of which...",EASY,"['math', 'dp', 'constructive algorithms']","['Dynamic programming', 'Mathematics', 'Constr...",['Dynamic programming'],https://codeforces.com/problemset/problem/626/B,NaN,NaN,NaN,NaN,Below is provided a **competitive programming*...,"['2\nRB\n', '3\nGRG\n', '5\nBBBBB\n', '1\nR\n'...","['G\n', 'BR\n', 'B\n', 'R\n', 'BGR\n', 'BG\n',...",False
2,Monocarp has been collecting rare magazines fo...,EASY,"['greedy', 'dp', 'constructive algorithms']","['Greedy algorithms', 'Dynamic programming', '...","['Dynamic programming', 'Greedy algorithms']",https://codeforces.com/problemset/problem/1743/C,NaN,NaN,2 seconds,NaN,Below is provided a **competitive programming*...,['4\n5\n01110\n10 5 8 9 6\n6\n011011\n20 10 9 ...,"['27\n80\n0\n14\n', '2\n2\n2\n2\n2\n2\n2\n', '...",False
3,Justin likes to play a game called “Weight of ...,EASY,"['dp & greedy', 'dp']",['Dynamic programming'],['Dynamic programming'],https://open.kattis.com/problems/weightofwords,NaN,NaN,NaN,NaN,Below is provided a **competitive programming*...,"['11 131\n', '7 96\n', '26 677\n']","['programming\n', 'contest\n', 'impossible\n']",False
4,Athenaeus has just finished creating his lates...,EASY,"['greedy', 'dp']","['Dynamic programming', 'Greedy algorithms']","['Dynamic programming', 'Greedy algorithms']",https://codeforces.com/problemset/problem/1466/B,NaN,NaN,1 second,NaN,Below is provided a **competitive programming*...,['5\n6\n1 2 2 2 5 6\n2\n4 4\n6\n1 1 3 4 4 5\n1...,"['5\n2\n6\n1\n3\n', '5\n2\n6\n1\n3\n', '5\n2\n...",False


In [134]:
final_df.loc[final_df['inputs'].isna()].sum()
# final_df.info()

question                    0
difficulty                  0
raw_tags                    0
tags                        0
skill_types                 0
url                         0
Expected Auxiliary Space    0
time_limit                  0
memory_limit                0
Expected Time Complexity    0
prompts                     0
inputs                      0
outputs                     0
done                        0
dtype: object

In [130]:
checkdf['prompts'].apply(len).sum()

np.int64(919556)

In [131]:
# Count total number of tokens (words) in the 'prompts' column of checkdf
total_words = checkdf['prompts'].str.split().apply(len).sum()
print(f"Total words in 'prompts': {total_words}")

# If you want to count characters (tokens for some LLMs), use:
total_chars = checkdf['prompts'].str.len().sum()
print(f"Total characters in 'prompts': {total_chars}")

Total words in 'prompts': 155437
Total characters in 'prompts': 919556


In [91]:
input_df = pd.read_csv('./input.csv')
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                300 non-null    int64  
 1   id                        300 non-null    int64  
 2   question                  300 non-null    object 
 3   difficulty                300 non-null    object 
 4   raw_tags                  300 non-null    object 
 5   tags                      300 non-null    object 
 6   skill_types               300 non-null    object 
 7   url                       300 non-null    object 
 8   Expected Auxiliary Space  0 non-null      float64
 9   time_limit                0 non-null      float64
 10  memory_limit              179 non-null    object 
 11  Expected Time Complexity  6 non-null      object 
 12  prompts                   300 non-null    object 
 13  inputs                    300 non-null    object 
 14  outputs   

In [95]:
full_unique = full_df[['question', 'name', 'source']].drop_duplicates(subset='question')
recovered_df = pd.merge(input_df, full_unique, on="question", how="left")
recovered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                300 non-null    int64  
 1   id                        300 non-null    int64  
 2   question                  300 non-null    object 
 3   difficulty                300 non-null    object 
 4   raw_tags                  300 non-null    object 
 5   tags                      300 non-null    object 
 6   skill_types               300 non-null    object 
 7   url                       300 non-null    object 
 8   Expected Auxiliary Space  0 non-null      float64
 9   time_limit                0 non-null      float64
 10  memory_limit              179 non-null    object 
 11  Expected Time Complexity  6 non-null      object 
 12  prompts                   300 non-null    object 
 13  inputs                    300 non-null    object 
 14  outputs   

In [98]:
recovered_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [101]:
recovered_df.info(), input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        300 non-null    int64  
 1   question                  300 non-null    object 
 2   difficulty                300 non-null    object 
 3   raw_tags                  300 non-null    object 
 4   tags                      300 non-null    object 
 5   skill_types               300 non-null    object 
 6   url                       300 non-null    object 
 7   Expected Auxiliary Space  0 non-null      float64
 8   time_limit                0 non-null      float64
 9   memory_limit              179 non-null    object 
 10  Expected Time Complexity  6 non-null      object 
 11  prompts                   300 non-null    object 
 12  inputs                    300 non-null    object 
 13  outputs                   300 non-null    object 
 14  done      

(None, None)

In [100]:
recovered_df.to_csv("./prompts/new_input.csv", index=False)